In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import numpy as np

In [ ]:
def get_data(year,table_no):
    # Define the URL of the website
    url = "https://www.teaboard.gov.in/WEEKLYPRICES/"+year

    # Send an HTTP GET request to the URL
    response = requests.get(url)

    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        # Parse the HTML content of the page
        soup = BeautifulSoup(response.text, 'html.parser')

        # Find the table(s) you want to scrape
        tables = soup.find_all('table')
        
        
        # Check if there are tables on the page
        if tables:
            name = soup.find(id ='contn_Annexure_'+("I"*(table_no+1))+'_text' )
            name = re.sub(' ','_',re.sub('.*\\>','',re.sub(' -.*','',str(name.contents[0]))))+str(year)
            # Choose the table you want (e.g., the first table)
            table = tables[int(table_no)]

            # Extract the table data into a list of lists
            table_data = []
            for row in table.find_all('tr'):
                row_data = [cell.get_text(strip=True) for cell in row.find_all(['th', 'td'])]
                table_data.append(row_data)

            # Create a DataFrame from the table data
            df = pd.DataFrame(table_data)

            # Save the DataFrame as a CSV file\
            print(f"Table data has been scraped and saved as {name}.csv")
            name = name+'_'+str(table_no)+'.csv'
            df.to_csv(name, index = False,header = False)
            return df

            
        else:
            print("No tables found on the page.")
            return 
    else:
        print("Failed to retrieve the web page. Status code:", response.status_code)
        return 

In [ ]:
a   = ['2021','2022','2023']
for b in a:
    for i in range(3):
        a = get_data(b,i)

In [2]:
name = 'WEEKLY_AVERAGE_PRICES_OF_CTC_LEAF_AND_ALL_DUST_SOLD_AT_INDIAN_AUCTION_DURING'
no ='0'

In [3]:
df1 = pd.read_csv(name+'2021_'+no+'.csv')

df2 = pd.read_csv(name+'2022_'+no+'.csv')

df3 = pd.read_csv(name+'2023_'+no+'.csv')

In [20]:
df = pd.concat([df1, df2, df3])

In [21]:
df

,Week Ending/Date,Kolkata,Guwahati,Siliguri,Jalpaiguri,mjunction,Cochin,Coonoor,Coimbatore,Tea Serve
0,25/12/2021,195.20(183.61),167.91(160.80),172.21(164.48),NS(NS),158.49(128.08),137.55(171.84),94.21(112.94),111.38(130.42),86.98(99.16)
1,18/12/2021,192.17(176.24),175.77(157.43),178.90(160.59),NS(NS),160.55(126.37),131.83(170.61),94.01(110.74),107.15(129.53),86.83(99.10)
2,11/12/2021,201.17(176.11),178.89(165.85),179.26(157.84),NS(NS),143.40(140.03),125.48(173.74),97.54(107.07),106.23(137.50),86.95(98.32)
3,04/12/2021,209.80(181.13),181.62(163.78),169.49(158.00),NS(NS),136.55(111.00),125.33(185.58),97.42(108.56),99.00(136.99),83.77(133.38)
4,27/11/2021,224.98(186.52),185.18(168.80),170.54(156.82),NS(NS),148.79(162.33),125.57(197.97),96.12(128.93),100.66(160.31),81.61(128.13)
...,...,...,...,...,...,...,...,...,...,...
35,04/02/2023,148.16(161.20),141.62(146.47),132.78(129.89),NS(NS),NS(120.42),151.29(140.12),118.64(102.76),129.83(114.67),113.39(91.26)
36,28/01/2023,153.55(NS),140.58(146.73),140.01(139.98),NS(NS),119.41(127.18),149.40(140.7),117.51(103.64),126.11(111.73),111.19(89.83)
37,21/01/2023,152.02(166.24),139.6(149.83),138.39(146.52),NS(NS),114.83(120.57),143.28(142.85),114.9(100.07),126.78(109.38),108.78(88.30)
38,14/01/2023,162.08(175.82),144.66(153.57),145.15(152.10),NS(NS),134.87(129.16),143.83(142.55),113.77(97.92),120.92(112.60),105.31(89.91)


In [22]:
df = df.replace('\\(.*|\\)','', regex = True)

In [23]:
df = df.replace('\\s.*','', regex = True)

In [24]:
df = df.replace('NS',np.nan)

In [25]:
df = df.replace(np.nan, 0)

In [26]:
df['Week Ending/Date'] = pd.to_datetime(df['Week Ending/Date'], dayfirst=True)

In [27]:
df

,Week Ending/Date,Kolkata,Guwahati,Siliguri,Jalpaiguri,mjunction,Cochin,Coonoor,Coimbatore,Tea Serve
0,2021-12-25,195.20,167.91,172.21,0.0,158.49,137.55,94.21,111.38,86.98
1,2021-12-18,192.17,175.77,178.90,0.0,160.55,131.83,94.01,107.15,86.83
2,2021-12-11,201.17,178.89,179.26,0.0,143.40,125.48,97.54,106.23,86.95
3,2021-12-04,209.80,181.62,169.49,0.0,136.55,125.33,97.42,99.00,83.77
4,2021-11-27,224.98,185.18,170.54,0.0,148.79,125.57,96.12,100.66,81.61
...,...,...,...,...,...,...,...,...,...,...
35,2023-02-04,148.16,141.62,132.78,0.0,0,151.29,118.64,129.83,113.39
36,2023-01-28,153.55,140.58,140.01,0.0,119.41,149.40,117.51,126.11,111.19
37,2023-01-21,152.02,139.6,138.39,0.0,114.83,143.28,114.9,126.78,108.78
38,2023-01-14,162.08,144.66,145.15,0.0,134.87,143.83,113.77,120.92,105.31


In [28]:
df.sort_values(by = 'Week Ending/Date', inplace = True)

In [29]:
df.tail()

,Week Ending/Date,Kolkata,Guwahati,Siliguri,Jalpaiguri,mjunction,Cochin,Coonoor,Coimbatore,Tea Serve
4,2023-09-09,214.96,192.84,157.91,0.0,164.7,130.83,88.58,100.87,81.23
3,2023-09-16,211.39,193.55,158.68,0.0,168.33,134.45,90.28,102.8,82.32
2,2023-09-23,218.51,196.87,157.39,0.0,164.1,131.13,90.58,103.72,82.03
1,2023-09-30,216.32,198.37,159.63,0.0,155.33,130.31,92.88,101.55,80.34
0,2023-10-07,212.05,199.55,161.22,0.0,149.85,129.97,94.57,102.29,83.50


In [30]:
numeric_columns = df.columns.difference(['Week Ending/Date'])

# Number of lags
num_lags = 4
window_size1 = 3
window_size2 = 4
window_size3 = 5
for col in numeric_columns:
    df[f"{col}_3_weeks_moving_avg"] = df[col].rolling(window=window_size1).mean()
    df[f"{col}_4_weeks_moving_avg"] = df[col].rolling(window=window_size2).mean()
    df[f"{col}_5_weeks_moving_avg"] = df[col].rolling(window=window_size3).mean()

    for lag in range(1, num_lags + 1):
        df[f"{col}_week_{lag}"] = df[col].shift(lag)

In [31]:
df

,Week Ending/Date,Kolkata,Guwahati,Siliguri,Jalpaiguri,mjunction,Cochin,Coonoor,Coimbatore,Tea Serve,...,Tea Serve_week_2,Tea Serve_week_3,Tea Serve_week_4,mjunction_3_weeks_moving_avg,mjunction_4_weeks_moving_avg,mjunction_5_weeks_moving_avg,mjunction_week_1,mjunction_week_2,mjunction_week_3,mjunction_week_4
52,2021-01-02,187.04,164.08,168.57,0.0,130.40,0,0,0,108.74,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
51,2021-01-09,183.43,167.99,167.60,0.0,129.87,171.59,134.05,137.28,117.34,...,NaN,NaN,NaN,NaN,NaN,NaN,130.40,NaN,NaN,NaN
50,2021-01-16,183.20,168.27,163.66,0.0,138.55,166.89,135.42,139.27,124.95,...,108.74,NaN,NaN,132.940000,NaN,NaN,129.87,130.40,NaN,NaN
49,2021-01-23,181.99,164.85,160.07,0.0,125.10,168.86,132.29,139.11,126.47,...,117.34,108.74,NaN,131.173333,130.9800,NaN,138.55,129.87,130.40,NaN
48,2021-01-30,184.65,160.29,152.18,0.0,136.28,167.83,132.06,142.04,122.69,...,124.95,117.34,108.74,133.310000,132.4500,132.040,125.10,138.55,129.87,130.40
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4,2023-09-09,214.96,192.84,157.91,0.0,164.7,130.83,88.58,100.87,81.23,...,81.13,82.43,82.69,163.216667,166.1775,163.078,156.64,168.31,175.06,150.68
3,2023-09-16,211.39,193.55,158.68,0.0,168.33,134.45,90.28,102.8,82.32,...,81.08,81.13,82.43,163.223333,164.4950,166.608,164.7,156.64,168.31,175.06
2,2023-09-23,218.51,196.87,157.39,0.0,164.1,131.13,90.58,103.72,82.03,...,81.23,81.08,81.13,165.710000,163.4425,164.416,168.33,164.7,156.64,168.31
1,2023-09-30,216.32,198.37,159.63,0.0,155.33,130.31,92.88,101.55,80.34,...,82.32,81.23,81.08,162.586667,163.1150,161.820,164.1,168.33,164.7,156.64


In [32]:
df.to_csv(name+'_2021_2023_lag_data.csv', index = False)

In [ ]:
data